## **<font color="red">Introduction to Conversational Context: Session, State, and Memory</font>**
- Meaningful, multi-turn conversations require agents to understand context. Just like humans, they need to recall the conversation history: what's been said and done to maintain continuity and avoid repetition. The **Agent Development Kit (ADK)** provides structured ways to manage this context through `Session`, `State`, and `Memory`.
### **Core Concepts**
- Think of different instances of your conversations with the agent as distinct **conversation threads**, potentially drawing upon **long-term knowledge**.
  1. `Session`: The Current Conversation Thread
     - Represents a _single_, ongoing _interaction_ between a user and your agent system.
     - Contains the chronological sequence of messages and actions taken by the agent (referred to `Events`) during that specific interaction.
     - A `Session` can also hold temporary data (`State`) relevant only during this conversation.
  2. `State` (`session.state`): Data within the Current Conversation
     - Data stored within a specific `Session`.
     - Used to manage information relevant only to the current, active conversation thread (e.g., items in a shopping cart during this chat, user preferences mentioned in this session).
  3. `Memory`: Searchable, Cross-Session Information
     - Represents a store of information that might span multiple past sessions or include external data sources.
     - It acts as a knowledge base the agent can search to recall information or context beyond the immediate conversation.
### **Managing Context: Services**
ADK provides services to manage these concepts:
1. `SessionService`: Manages the different conversation threads (`Session` objects)
   - Handles the lifecycle: creating, retrieving, updating (appending `Events`, modifying `State`), and deleting individual `Session`.
2. `MemoryService`: Manages the Long-Term Knowledge Store (`Memory`)
   - Handles ingesting information (often from completed `Session`) into the long-term store.
   - Provides methods to search this stored knowledge based on queries.
- **Implementations:** ADK offers different implementations for both `SessionService` and `MemoryService`, allowing you to choose the storage backend that best fits your application's needs. Notably, **in-memory implementations** are provided for both services; these are designed specifically for **local testing and fast development**. It's important to remember that **all data stored using these in-memory options (sessions, state, or long-term knowledge) is lost when your application restarts**. For persistence and scalability beyond local testing, ADK also offers cloud-based and database service options.

## **<font color="red">Session: Tracking Individual Conversations</font>**
### **The `Session` Object**
- When a user starts interacting with your agent, the `SessionService` creates a `Session` object (`google.adk.sessions.Session`). This object acts as the container holding everything related to that one specific chat thread. Here are its key properties:
  - **Identification (`id`, `appName`, `userId`):** Unique labels for the conversation.
  - `id`: A unique identifier for this specific conversation thread, essential for retrieving it later. A SessionService object can handle multiple `Session`. This field identifies which aprticualr session object are we referring to. For example, "test_id_modification".
  - `app_name`: Identifies which agent application this conversation belongs to. For example, "id_modifier_workflow".
  - `userId`: Links the conversation to a particualr user.
- **History (`events`):** A chronological sequence of all interactions (Event objects – user messages, agent responses, tool actions) that have occurred within this specific thread.
- **Session State (`state`):** A place to store temporary data relevant only to this specific, ongoing conversation. This acts as a scratchpad for the agent during the interaction. We will cover how to use and manage `state` in detail in the next section.
- **Activity Tracking (`lastUpdateTime`):** A timeestamp indicating the last time an event occurred in this conversation thread.


In [2]:
from google.adk.sessions import InMemorySessionService, Session
import os
from config import config
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

# Create a simple session to examine its properties
temp_service = InMemorySessionService()
example_session = await temp_service.create_session(
 app_name="my_app",
 user_id="example_user",
 state={"initial_key": "initial_value"} # State can be initialized
)

print(f"--- Examining Session Properties ---")
print(f"ID (`id`):                {example_session.id}")
print(f"Application Name (`app_name`): {example_session.app_name}")
print(f"User ID (`user_id`):         {example_session.user_id}")
print(f"State (`state`):           {example_session.state}") # Note: Only shows initial state here
print(f"Events (`events`):         {example_session.events}") # Initially empty
print(f"Last Update (`last_update_time`): {example_session.last_update_time:.2f}")
print(f"---------------------------------")

# Clean up (optional for this example)
temp_service = await temp_service.delete_session(app_name=example_session.app_name,
                         user_id=example_session.user_id, session_id=example_session.id)
print("The final status of temp_service - ", temp_service)


--- Examining Session Properties ---
ID (`id`):                6da1005a-06d1-4d70-b24e-88049ce3aea6
Application Name (`app_name`): my_app
User ID (`user_id`):         example_user
State (`state`):           {'initial_key': 'initial_value'}
Events (`events`):         []
Last Update (`last_update_time`): 1771214580.65
---------------------------------
The final status of temp_service -  None


In [5]:
import os
import asyncio
from config import config

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types


# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "my_chatbot_app"
USER_ID = "user_1"
SESSION_ID = "session_001"


# -------------------------------------------------
# Create Agent
# -------------------------------------------------
agent = LlmAgent(
    name="assistant",
    model="gemini-2.5-flash",
    instruction="""
You are a helpful AI assistant.
Keep responses clear and conversational.
"""
)


# -------------------------------------------------
# Chatbot Conversation Function
# -------------------------------------------------

# 1️⃣ Create Session Service
session_service = InMemorySessionService()

# 2️⃣ Create Session
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"conversation_started": True}
)

# 3️⃣ Create Runner
runner = Runner(
    agent=agent,
    session_service=session_service,
    app_name=APP_NAME
)

print("\n================ CHATBOT STARTED ================\n")

# # 4️⃣ Simulated Conversation
# user_messages = [
#     "Hello!",
#     "What can you help me with?",
#     "Explain Artificial Intelligence in simple words.",
#     "Give me a short example."
# ]

# for message in user_messages:

#     print(f"\n👤 User: {message}")

#     # Send message to agent
#     async for event in runner.run_async(
#         user_id=USER_ID,
#         session_id=SESSION_ID,
#         new_message=types.Content(role="user", parts=[types.Part(text=message)])
#     ):
#         if event.content and event.content.parts:
#             response_text = event.content.parts[0].text
#             print(f"🤖 Assistant: {response_text}")

while True:
    message = input("👤 User:")
    # Stop loop
    if message in [None, '', "exit", "quit"]:
        break

    # Send message to agent
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=types.Content(role="user", parts=[types.Part(text=message)])
    ):
        if event.content and event.content.parts:
            response_text = event.content.parts[0].text
            print(f"🤖 Assistant: {response_text}")
    print('\n')

print("\n================ CHATBOT ENDED ================\n")

# 5️⃣ Inspect Final Session
final_session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print("\n--- Final Session State ---")
print("Session ID:", final_session.id)
print("State:", final_session.state)
print("Total Events:", len(final_session.events))
print("---------------------------------\n")

# 6️⃣ Clean Up
await session_service.delete_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print("✅ Session deleted successfully.")



================ CHATBOT STARTED ================



👤 User: Hi


🤖 Assistant: Hi there! How can I help you today?




👤 User: What is quantum physics


🤖 Assistant: Quantum physics is a fascinating branch of physics that studies the *very, very small* – things like atoms and the particles that make them up (electrons, protons, neutrons, photons, etc.).

Think of it this way:

*   **Classical physics** (like the physics Newton developed) explains how big things work – planets orbiting the sun, a ball rolling down a hill, a car driving. It works great for our everyday world.
*   **Quantum physics** takes over when you zoom in to the incredibly tiny scale. At this level, the rules change drastically, and particles behave in ways that seem really strange and counter-intuitive compared to our everyday experience.

Here are some of the mind-bending core ideas:

1.  **Quantization:** Unlike classical physics where energy, momentum, etc., can have any value, in the quantum world, these properties often come in discrete "packets" or "chunks" called **quanta**. Imagine a ramp versus stairs – classical physics is like a ramp where you can stop a

👤 User: what is quantum tunnel


🤖 Assistant: Ah, quantum tunneling! This is one of those phenomena that really highlights how strange the quantum world is compared to our everyday experience.

Let's start with a classical analogy:

Imagine you have a **ball** and a **hill**. If you roll the ball towards the hill, it needs to have enough energy to get over the top. If it doesn't have enough energy, it will just roll part-way up and then roll back down. It cannot magically appear on the other side of the hill without having the energy to surmount it. That's common sense, right?

Now, enter the quantum world:

**Quantum tunneling** is like if that ball, *without having enough energy to get over the hill*, suddenly **appears on the other side of the hill**!

How does this happen?

1.  **Wave-Particle Duality:** Remember that quantum particles (like electrons) can also behave like waves? When a particle approaches a "barrier" (like our hill, or a region of higher energy), its wave function (which describes the probability

👤 User: 



================ CHATBOT ENDED ================


--- Final Session State ---
Session ID: session_001
State: {'conversation_started': True}
Total Events: 6
---------------------------------

✅ Session deleted successfully.


### **Managing Sessions with a SessionService**
- As seen above, you don't typically create or manage Session objects directly. Instead, you use a SessionService. This service acts as the central manager responsible for the entire lifecycle of your conversation sessions.
- Its core responsibilities include:
  - **Starting New Conversations:** Creating fresh Session objects when a user begins an interaction.
  - **Resuming Existing Conversations:** Retrieving a specific Session (using its ID) so the agent can continue where it left off.
  - **Saving Progress:** Appending new interactions (Event objects) to a session's history. This is also the mechanism through which session state gets updated (more in the State section).
  - **Listing Conversations:** Finding the active session threads for a particular user and application.
  - **Cleaning Up:** Deleting Session objects and their associated data when conversations are finished or no longer needed.

### **`SessionService` Implementations**
- ADK provides different `SessionService` implementations, allowing you to choose the storage backend that best suits your needs:
- `InMemorySessionService`
  - **How it works:** Stores all session data directly in the application's memory.
  - **Persistence:** None. All conversation data is lost if the application restarts.
  - **Requires:** Nothing extra.
  - **Best for:** Quick development, local testing, examples, and scenarios where long-term persistence isn't required.
### **`VertexAiSessionService`**
- **How it works:** Uses Google Cloud Vertex AI infrastructure via API calls for session management.
- **Persistence:** Yes. Data is managed reliably and scalably via Vertex AI Agent Engine.
- **Requires:**
  - A Google Cloud project (pip install vertexai)
  - A Google Cloud storage bucket that can be configured by this step.
  - A Reasoning Engine resource name/ID that can setup following this tutorial.
  - If you do not have a Google Cloud project and you want to try the VertexAiSessionService, see Vertex AI Express Mode.
- **Best for:** Scalable production applications deployed on Google Cloud, especially when integrating with other Vertex AI features.

In [2]:
# Requires: pip install google-adk[vertexai]
# Plus GCP setup and authentication
from google.adk.sessions import VertexAiSessionService

PROJECT_ID = "your-gcp-project-id"
LOCATION = "us-central1"
# The app_name used with this service should be the Reasoning Engine ID or name
REASONING_ENGINE_APP_NAME = "projects/your-gcp-project-id/locations/us-central1/reasoningEngines/your-engine-id"

session_service = VertexAiSessionService(project=PROJECT_ID, location=LOCATION)
# Use REASONING_ENGINE_APP_NAME when calling service methods, e.g.:
# session_service = await session_service.create_session(app_name=REASONING_ENGINE_APP_NAME, ...)

### **`DatabaseSessionService`**
- **How it works:** Connects to a relational database (e.g., _PostgreSQL_, _MySQL_, _SQLite_) to store session data persistently in tables.
- **Persistence:** Yes. Data survives application restarts.
- **Requires:** A configured database.
- **Best for:** Applications needing reliable, persistent storage that you manage yourself.

In [3]:
from google.adk.sessions import DatabaseSessionService
# Example using a local SQLite file:
# Note: The implementation requires an async database driver.
# For SQLite, use 'sqlite+aiosqlite' instead of 'sqlite' to ensure async compatibility.
db_url = "sqlite+aiosqlite:///./my_agent_data.db"
session_service = DatabaseSessionService(db_url=db_url)

In [9]:
import os
import asyncio
from config import config

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService
from google.genai import types


# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "db_chatbot_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

# SQLite async URL
db_url = "sqlite+aiosqlite:///./db_agent_data.db"


# -------------------------------------------------
# Create Agent
# -------------------------------------------------
agent = LlmAgent(
    name="assistant",
    model="gemini-2.5-flash",
    instruction="""
You are a helpful AI assistant.
Keep responses clear and conversational.
"""
)


# -------------------------------------------------
# Main Async Function
# -------------------------------------------------

# 1️⃣ Create Database Session Service
session_service = DatabaseSessionService(db_url=db_url)

# 2️⃣ Create Session
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"conversation_started": True}
)

# 3️⃣ Create Runner
runner = Runner(
    agent=agent,
    session_service=session_service,
    app_name=APP_NAME
)

print("\n================ CHATBOT STARTED ================\n")

# 4️⃣ Interactive Conversation
while True:

    message = input("👤 User: ")

    if message.lower() in ["", "exit", "quit"]:
        break

    print("🤖 Assistant: ", end="", flush=True)

    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )
    ):
        if event.content and event.content.parts:
            print(event.content.parts[0].text, end="", flush=True)

    print("\n")

print("\n================ CHATBOT ENDED ================\n")

# 5️⃣ Inspect Final Session
final_session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print("\n--- Final Session State ---")
print("Session ID:", final_session.id)
print("State:", final_session.state)
print("Total Events:", len(final_session.events))
print("---------------------------------\n")



================ CHATBOT STARTED ================



👤 User:  Hello


🤖 Assistant: Hello there! How can I help you today?



👤 User:  What is dual nature of light?


🤖 Assistant: That's a fantastic question and a cornerstone of modern physics!

The **dual nature of light** (also known as wave-particle duality) means that light isn't just one thing – it behaves like both a **wave** and a **particle**, depending on how you observe or measure it.

Let's break it down:

1.  **Light as a Wave:**
    *   For centuries, scientists observed phenomena like **diffraction** (light bending around corners) and **interference** (light waves combining to create patterns of bright and dark fringes). These effects are perfectly explained by treating light as an electromagnetic wave, much like ripples in water or sound waves.
    *   Wave theory explains things like color (different wavelengths), reflection, refraction, and polarization.

2.  **Light as a Particle:**
    *   In the early 20th century, some experiments couldn't be explained by wave theory alone. The most famous example is the **photoelectric effect**, where light shining on a metal surface can eject 

...

🤖 Assistant: The most direct and compelling way to demonstrate the particle-like nature of light is through the **photoelectric effect**. This is the phenomenon where light shining on a metal surface causes electrons to be ejected from that surface.

While Thomas Young's experiment elegantly showed light's wave nature, the photoelectric effect (and its explanation by Albert Einstein, building on Max Planck's ideas) was what truly solidified the idea that light also behaves like discrete particles, which we now call **photons**.

Here's how it works and why it demonstrates particle nature:

### The Photoelectric Effect: A Demonstration of Light as Particles

**The Setup (Conceptual):**

Imagine a vacuum tube containing two metal plates. One plate (the cathode) is exposed to light, and the other (the anode) is positioned to collect any ejected electrons. A circuit with a sensitive ammeter measures any current flow (indicating electrons are being ejected). You can also apply a voltage to 

👤 User:  Is there any single experiment which can describe the both?


🤖 Assistant: That's an excellent follow-up question, and it really gets to the heart of the "duality" concept in quantum mechanics!

The challenge is that whenever you design an experiment to *detect* the wave nature of light, you usually don't see its particle nature simultaneously, and vice-versa. It's like light "chooses" which behavior to exhibit based on how you're looking at it.

However, the closest we can get to a single experiment that beautifully demonstrates *both* aspects is the **Modern (Single-Photon) Double-Slit Experiment**.

Here's how it works and what it shows:

### The Single-Photon Double-Slit Experiment

This is an extension of Young's original experiment, but with a crucial difference:

**The Setup:**

1.  **Extremely Dim Light Source:** Instead of a continuous beam, you use a light source so dim that it emits **one photon at a time**.
2.  **Double Slits:** The same setup with two narrow, parallel slits.
3.  **Sensitive Detector Screen:** A screen capable of dete

👤 User:  exit



================ CHATBOT ENDED ================


--- Final Session State ---
Session ID: session_001
State: {'conversation_started': True}
Total Events: 10
---------------------------------

